In [1]:
from framenet import builder, ecg
from itertools import chain 

fn, fnb = builder.build()

In [2]:
fs = fn.typesystem.toposort()

In [3]:
fs[0:2]

[Frame(name=Event, ID=187, relations=[Is Inherited by: ['Change_of_consistency', 'Rotting', 'Process_resume', 'Process_start', 'Process_stop', 'Experience_bodily_harm', 'Becoming', 'Misdeed', 'Process_end', 'Intentionally_act', 'Coming_to_believe', 'Getting', 'Eventive_affecting', 'Process_pause', 'Waking_up', 'Ceasing_to_be', 'Go_into_shape', 'Objective_influence', 'Motion', 'Come_together', 'Birth_scenario', 'Coming_to_be', 'Change_position_on_a_scale', 'Coincidence', 'Traversing', 'Emergency', 'Losing_someone'], Is Used by: ['Participation', 'Preventing', 'Hindering', 'Frequency', 'Temporal_pattern', 'Locale_by_event', 'Manner'], Subframe of: ['Change_of_state_scenario'], Precedes: ['Change_of_state_endstate'], Is Preceded by: ['Change_of_state_initial_state']], elements=[FrameElement(frame_name=Event, name=Event, ID=1443, coreType=Core-Unexpressed, semtype=State_of_affairs), FrameElement(frame_name=Event, name=Place, ID=1444, coreType=Core, semtype=Locative_relation), FrameElement(

In [4]:
[(f.name, f.parents, f.children) for f in fs[:2]]

[('Event', [], []), ('Intentionally_act', ['Event'], [])]

In [5]:
aa = fn.typesystem.get_ancestors(fs[10])
[(a.name, a.parents) for a in aa]

[('State', []),
 ('Relation', []),
 ('Trajector-Landmark', ['Relation']),
 ('Locative_relation', ['State', 'Trajector-Landmark']),
 ('Abounding_with', ['Locative_relation'])]

In [9]:
import pandas as pd
import qgrid

from itertools                import chain
from framenet.util            import aget, curry, flatten, force, take, dfs_iterator, flatmap
from framenet.ecg.generation  import frame_element_relations, frames
from pprint                   import pprint, pformat

def diff(xs, ys):
    return [x for x in xs if x not in ys]

def incl(xs, ys):
    return [(x, x in ys) for x in xs]

fmt   = aget('name', 'coreType', 'frame', default='<unk>')
names = lambda es: [e.name for e in es]

@curry
def inherited(frame):
    return [r for r in frame.fe_relations 
            if r.subFrame == frame.name and r.name.startswith('Inh')]

@curry
def inherited_fes(fn, fers, frame):
    """Returns inherited FE IDs using FE relations.
    """
    return [(fer['supID'], fer['relationSupID']) 
            for fer in fers 
            if fer['relationType'] == 'Inheritance' 
               and fer['relationSubID'] == frame.ID]
    
ancestors = fn.typesystem.get_ancestors

def unique(xs):
    marked = set()
    ys = []
    for x in xs:
        if x not in marked:
            ys.append(x)
            marked.add(x)
    return ys

@curry
def inherited_elements(src_frames):
    return unique(flatten(p.elements for p in src_frames))

@curry
def fe_parents(fers, feID):
    return fers[fers.subID == feID] 

@curry
def fe_ancestors(fers, feID):
    return dfs_iterator(set(), fe_parents(fers), lambda fe: fe.ID, feID)
    
@curry
def inherited_elements2(fers, src_frames):
    pp = fe_parents(fers)
    return unique(flatten(pp(fe) for fe in src_frames))

In [10]:
fers = pd.DataFrame(frame_element_relations())
qgrid.show_grid(fers, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

In [11]:
motion_ancestors = fn.typesystem.get_ancestors(fn.get_frame('Cause_motion'))
es = inherited_elements(motion_ancestors)
pd.DataFrame([fmt(e) for e in es])

,0,1,2
0,Event,Core-Unexpressed,Event
1,Place,Core,None
2,Explanation,Extra-Thematic,None
3,Time,Core,None
4,Duration,Extra-Thematic,None
5,Manner,Peripheral,Manner
6,Frequency,Extra-Thematic,Frequency
7,Timespan,Extra-Thematic,Timespan
8,Influencing_entity,Core,None
9,Dependent_entity,Core,None


In [12]:
fers[fers.subID == 1610]

,ID,relationID,relationSubFrameName,relationSubID,relationSupID,relationSuperFrameName,relationType,subFEName,subID,supID,superFEName


In [13]:
fes = list(frames())
fes_df = pd.DataFrame(list(fes))

# import qgrid
# qgrid.show_grid(fes_df)

fe_by_ID = {fe['ID']: fe for fe in fes}

ii1 = inherited_fes(fn, frame_element_relations(), fs[1])
print(len(ii1))
pprint([fe_by_ID[i]['name'] for i, _ in ii1])
inherited_elements(fs[1].get_parents(fn))

5
['Frequency', 'Manner', 'Place', 'Time', 'Event']


[FrameElement(frame_name=Event, name=Event, ID=1443, coreType=Core-Unexpressed, semtype=State_of_affairs),
 FrameElement(frame_name=Event, name=Place, ID=1444, coreType=Core, semtype=Locative_relation),
 FrameElement(frame_name=Event, name=Explanation, ID=1445, coreType=Extra-Thematic, semtype=State_of_affairs),
 FrameElement(frame_name=Event, name=Time, ID=1446, coreType=Core, semtype=Time),
 FrameElement(frame_name=Event, name=Duration, ID=4960, coreType=Extra-Thematic, semtype=Duration),
 FrameElement(frame_name=Event, name=Manner, ID=4961, coreType=Peripheral, semtype=Manner),
 FrameElement(frame_name=Event, name=Frequency, ID=4962, coreType=Extra-Thematic, semtype=None),
 FrameElement(frame_name=Event, name=Timespan, ID=16172, coreType=Extra-Thematic, semtype=None)]

In [14]:
pp = fe_parents(frame_element_relations())
inherited_elements2(fe_by_ID[198], pp(fe_by_ID[198]))

AttributeError: 'list' object has no attribute 'subID'

In [ ]:
ee1 = fn.get_frame_from_id(198).elements
print(len(ee1))
pprint(fn.get_frame('Event').elements)
ee1

In [ ]:
ancestors(frame_element_relations(), fe_by_ID['11991'])

In [ ]:
# pprint(fs[1].elements)
pprint(inherited(fs[1]))
pprint(inherited_elements(fs[1], fs[1].get_parents(fn)))

## <u>To do</u>

1. Do this for `Cause_motion` (and `Fluidic_motion`)
1. `FE` <--> `FE` relations for `Transive_action`, `Cause_motion`, `Fluidic_motion`, `Passing` 
1. ECG schema for `Cause_motion`
1. There is no 4th thing! :-D

In [ ]:
def check(fn, i, frame):
    ancestors     = fn.typesystem.get_ancestors
    inherited_es  = set(flatten(p.elements for p in ancestors(frame)))
    elements      = set(frame.elements)
    
    d1            = incl(inherited_es, elements)
    not_in        = [(e.name, inherited(frame, e)) for e, in_ in d1 if not in_]
    assert not not_in, '%s (%d):\n%s' % (frame.name, i, pformat(not_in))
    
#     d2 = [fmt(e) for e in diff(ns, all_elements)]
#     assert not d1, '%s [%d]: in ancestors, but not in frame: %s' % (frame.name, i, pformat(d1))
#     assert not d2, '%s [%d]: in frame, but not in ancestors: %s' % (frame.name, i, pformat(d2))
    return not not_in #and not d2

In [ ]:
all(check(fn, i, fr) for i, fr in enumerate(fs))

In [ ]:
list(fn.typesystem.get_ancestors(fs[4]))

In [ ]:
rr = [r for r in fs[4].fe_relations]
print(len(rr))
print(fs[4].name, fs[4].parents)
print(fs[4].name)
rr

In [ ]:
sorted(inherited_elements(fs[4], ancestors(fs[4])), key=lambda e: (e.frame_name, e.name)) 

In [ ]:
pp = list(fs[4].get_parents(fn))

In [ ]:
pp[0].elements

In [ ]:
fs[4].elements.count

In [ ]:
[(e.name, e.coreType, name(e.semtype), e.frame) 
 for e in fs[4].elements]

In [ ]:
[(a.name, [fmt(e) for e in a.elements]) for a in fn.typesystem.get_ancestors(fs[4])]

In [ ]:
[e for e in fs[1].elements if e not in fs[0].elements]

In [ ]:
[(r.fe1, r.name, r.fe2) for r in fs[4].fe_relations]

In [ ]:
fs[4].fe_relations[0].subFrame

In [ ]:
[e for e in fs[2].elements]

In [ ]:
fnb.build_lus_for_frame(fs[2].name, fn)

In [ ]:
fs[2].individual_valences[0].__class__.__name__

In [ ]:
fs[2].fe_realizations[0].lexeme

In [ ]:
import os
from os.path import join
from framenet.semtypes import read_semtypes, format_semtype_to_ontology

sts = read_semtypes(join(os.getenv('FN_HOME'), 'semTypes.xml'))
len(sts)

In [ ]:
for st in sts[:10]:
    print(format_semtype_to_ontology(st))